In [1]:
import pandas as pd
import numpy as np
from itertools import combinations, product

In [2]:
# read in manually updated csvs
drivers = pd.read_csv('f1 - drivers.csv')
constructors = pd.read_csv('f1 - constructors.csv')

In [3]:
# points per position: how many points does each race position score, on average?
ppp = drivers.sort_values('Season Points per Race',ascending=False)['Season Points per Race']

In [4]:
# expected finish order
efo = drivers.sort_values(['Implied Win Probability','Season Points per Race'],ascending=False)[['Driver','Season Points per Race','Cost','Constructor']]

In [5]:
# expected points (weighting is untested)
efo['Avg. Points per Position'] = list(ppp)
efo['Expected Points'] = 0.75*efo['Avg. Points per Position'] + 0.25*efo['Season Points per Race']
drivers = efo

In [6]:
# list every combination of 5 drivers 'dri'
dri = []
for driver_combo in combinations(drivers['Driver'].values,5):
    dri.append(driver_combo)

In [7]:
# list every combination of 2 constructors 'con'
con = []
for con_combo in combinations(constructors['Team'].values,2):
    con.append(con_combo)

In [8]:
# list every combination of those dri and con 'lineup'
lineup = []
for lineup_product in product(dri,con):
    lineup.append(lineup_product)

In [9]:
# pull the lineups into a dataframe
rows = []
for entry in lineup:
    d1 = entry[0][0]
    d2 = entry[0][1]
    d3 = entry[0][2]
    d4 = entry[0][3]
    d5 = entry[0][4]
    c1 = entry[1][0]
    c2 = entry[1][1]
    row = [d1,d2,d3,d4,d5,c1,c2]
    rows.append(row)
    
cols = ['D1','D2','D3','D4','D5','C1','C2']
df = pd.DataFrame(rows,columns=cols)

In [10]:
# map costs back onto dataframe
driver_map = drivers[['Driver','Cost']].set_index('Driver')['Cost'].to_dict()

for col in ['D1','D2','D3','D4','D5']:
    new_col_str = col + '_budget'
    df[new_col_str] = df[col].map(driver_map)

con_map = constructors[['Team','Cost']].set_index('Team')['Cost'].to_dict()

for col in ['C1','C2']:
    new_col_str = col + '_budget'
    df[new_col_str] = df[col].map(con_map)

df['total_budget'] = df[['D1_budget','D2_budget','D3_budget','D4_budget','D5_budget','C1_budget','C2_budget']].sum(axis=1)

In [11]:
# sum constructor expected points from its drivers
constructors['Expected Points'] = constructors['Team'].map(drivers.groupby('Constructor')['Expected Points'].sum())

In [12]:
# map win probabilities back onto dataframe
driver_map = drivers[['Driver','Expected Points']].set_index('Driver')['Expected Points'].to_dict()

for col in ['D1','D2','D3','D4','D5']:
    new_col_str = col + '_xp'
    df[new_col_str] = df[col].map(driver_map)

con_map = constructors[['Team','Expected Points']].set_index('Team')['Expected Points'].to_dict()

for col in ['C1','C2']:
    new_col_str = col + '_xp'
    df[new_col_str] = df[col].map(con_map)

# the top driver you assign your 'DRS Boost' to gets their score doubled
df['D1_xp'] = df['D1_xp']*2

df['total_xp'] = df[['D1_xp','D2_xp','D3_xp','D4_xp','D5_xp','C1_xp','C2_xp']].sum(axis=1)

In [13]:
# reset index
df = df.reset_index()

In [14]:
# list the remaining lineups by combined expected points
drop = ['D1_xp', 'D2_xp', 'D3_xp', 'D4_xp','D5_xp', 'C1_xp', 'C2_xp','D1_budget','D2_budget', 'D3_budget', 'D4_budget', 'D5_budget', 'C1_budget','C2_budget']
df.drop(columns=drop,inplace=True)

In [15]:
# sort by winningest
df.sort_values(['total_xp','total_budget'],ascending=[False,True],inplace=True)

In [16]:
def best_available_lineups(df,team:list,budget:float,free_transfers:int):
    '''
    given a dataframe of upcoming race probs and your team info, spits out the best lineups to switch to
    
    df: pandas dataframe of all possible lineups, with corresponding total_budget and total_wp
    team_n: a list of the team's current entrants
    budget: the team's current cost cap
    free_transfers: the number of free transfers available
    
    '''
    df = df[df['total_budget'] <= budget] # filter by budget

    # subset to teams that share enough entrants with the current entry
    # to stay under the free transfer limit
    t1 = set(team)
    match_indices = [] # empty list to store match indices
    for possible_entry in df.values:
        t2 = set(possible_entry)
        if len(t1&t2) >= (7 - free_transfers):
            match_indices.append(possible_entry[0])
    df = df[df['index'].isin(match_indices)]
    return df

In [20]:
# team 1
team_1 = ['Max Verstappen','Yuki Tsunoda','Carlos Sainz','Fernando Alonso','Isack Hadjar','McLaren','Racing Bulls']
t1_new = best_available_lineups(df,team_1,budget=10000,free_transfers=15)
t1_new.head()

,index,D1,D2,D3,D4,D5,C1,C2,total_budget,total_xp
45,45,Max Verstappen,Lando Norris,Oscar Piastri,George Russell,Lewis Hamilton,McLaren,Ferrari,198.9,239.400
0,0,Max Verstappen,Lando Norris,Oscar Piastri,George Russell,Charles Leclerc,McLaren,Ferrari,199.4,239.400
46,46,Max Verstappen,Lando Norris,Oscar Piastri,George Russell,Lewis Hamilton,McLaren,Red Bull,197.6,239.125
1,1,Max Verstappen,Lando Norris,Oscar Piastri,George Russell,Charles Leclerc,McLaren,Red Bull,198.1,239.125
47,47,Max Verstappen,Lando Norris,Oscar Piastri,George Russell,Lewis Hamilton,McLaren,Mercedes,194.9,239.100


In [22]:
# team 2
team_2 = ['Oscar Piastri','Pierre Gasly','Isack Hadjar','Fernando Alonso','Gabriel Bortoleto','McLaren','Red Bull']
t2_new = best_available_lineups(df,team_2,budget=117.4,free_transfers=2)
t2_new.head()

,index,D1,D2,D3,D4,D5,C1,C2,total_budget,total_xp
387496,387496,Oscar Piastri,Fernando Alonso,Carlos Sainz,Nico Hulkenberg,Isack Hadjar,McLaren,Red Bull,116.3,167.550
387901,387901,Oscar Piastri,Fernando Alonso,Carlos Sainz,Isack Hadjar,Gabriel Bortoleto,McLaren,Red Bull,116.8,166.225
387902,387902,Oscar Piastri,Fernando Alonso,Carlos Sainz,Isack Hadjar,Gabriel Bortoleto,McLaren,Mercedes,114.1,166.200
391546,391546,Oscar Piastri,Fernando Alonso,Nico Hulkenberg,Isack Hadjar,Gabriel Bortoleto,McLaren,Red Bull,117.3,165.775
391547,391547,Oscar Piastri,Fernando Alonso,Nico Hulkenberg,Isack Hadjar,Gabriel Bortoleto,McLaren,Mercedes,114.6,165.750


In [23]:
# team 3
team_3 = ['Max Verstappen','Liam Lawson','Yuki Tsunoda','Fernando Alonso','Isack Hadjar','McLaren','Racing Bulls']
t3_new = best_available_lineups(df,team_3,budget=109.1,free_transfers=15)
t3_new.head()

,index,D1,D2,D3,D4,D5,C1,C2,total_budget,total_xp
470072,470072,George Russell,Fernando Alonso,Carlos Sainz,Isack Hadjar,Pierre Gasly,McLaren,Mercedes,108.9,153.850
470117,470117,George Russell,Fernando Alonso,Carlos Sainz,Isack Hadjar,Franco Colapinto,McLaren,Mercedes,108.9,153.750
142698,142698,Max Verstappen,Fernando Alonso,Carlos Sainz,Nico Hulkenberg,Isack Hadjar,McLaren,Williams,107.9,152.625
142253,142253,Max Verstappen,Fernando Alonso,Carlos Sainz,Alex Albon,Nico Hulkenberg,McLaren,Kick Sauber,107.1,152.025
280398,280398,Lando Norris,Fernando Alonso,Carlos Sainz,Nico Hulkenberg,Isack Hadjar,McLaren,Williams,109.1,152.025
